In [ ]:
import os
from getpass import getpass

import openai

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
openai.api_key = openai_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key

In [ ]:
import pandas as pd
import phoenix as px
from phoenix.trace import TraceDataset, using_project


# Display the complete contents of dataframe cells.
# pd.set_option("display.max_colwidth", None)

In [ ]:
testset_hex = "6d71d4a4-55cf-4222-a084-b988481f296b" 

In [ ]:
with using_project("ragas-testset"):
    trace_dataset = TraceDataset.load(testset_hex, directory='./data')
    session = px.launch_app(trace=trace_dataset)

In [ ]:
from llama_index.core import SimpleDirectoryReader

dir_path = "./data/prompt-engineering-papers"
reader = SimpleDirectoryReader(dir_path, num_files_limit=2)
documents = reader.load_data()

In [ ]:
len(documents)

In [ ]:
documents[1]

Set handler for `Llama Index` and `Langchain`(Ragas uses Langchain)

In [ ]:

from llama_index.core import set_global_handler
from phoenix.trace.langchain import LangChainInstrumentor


# Setup instrumentation for both llama-index and LangChain (used by Ragas)
set_global_handler("arize_phoenix")
LangChainInstrumentor().instrument()

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from phoenix.trace import using_project


def build_query_engine(documents):
    vector_index = VectorStoreIndex.from_documents(
        documents,
        embed_model=OpenAIEmbedding(),
    )
    query_engine = vector_index.as_query_engine(similarity_top_k=2)
    return query_engine


with using_project("indexing"):
    # By assigning a project name, the instrumentation will send all the embeddings to the indexing project
    query_engine = build_query_engine(documents)

In [ ]:
my_traces = px.Client().get_trace_dataset(project_name="indexing").save(directory="./data")

In [ ]:
my_traces.hex

In [ ]:
px.Client().query_spans(project_name="indexing").head(n=1)


In [ ]:
px.Client().query_spans(project_name="indexing")["attributes.embedding.embeddings"]

Embeddings to dataframe for use in visualization

In [ ]:
from phoenix.trace.dsl.helpers import SpanQuery

client = px.Client()
corpus_df = px.Client().query_spans(
    SpanQuery().explode(
        "embedding.embeddings",
        text="embedding.text",
        vector="embedding.vector",
    ),
    project_name="indexing",
)
corpus_df.head()

In [ ]:
test_df = pd.read_csv(f"./data/{testset_hex}-testset.csv")
test_df.head()

In [ ]:
import pandas as pd
from datasets import Dataset
from phoenix.trace import using_project
from tqdm.auto import tqdm


def generate_response(query_engine, question):
    response = query_engine.query(question)
    return {
        "answer": response.response,
        "contexts": [c.node.get_content() for c in response.source_nodes],
    }


def generate_ragas_dataset(query_engine, test_df):
    test_questions = test_df["question"].values
    responses = [generate_response(query_engine, q) for q in tqdm(test_questions)]

    dataset_dict = {
        "question": test_questions,
        "answer": [response["answer"] for response in responses],
        "contexts": [response["contexts"] for response in responses],
        "ground_truth": test_df["ground_truth"].values.tolist(),
    }
    ds = Dataset.from_dict(dataset_dict)
    return ds


with using_project("llama-index"):
    ragas_eval_dataset = generate_ragas_dataset(query_engine, test_df)

ragas_evals_df = pd.DataFrame(ragas_eval_dataset)
ragas_evals_df.head()